# Necessary imports

In [1]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.7 MB/s eta 0:00:00


In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque, namedtuple
import random
from torch.functional import F
import datetime

# Constants

In [3]:
LR = 0.001
BUFFER_SIZE = 100_000
BATCH_SIZE = 64
GAMMA = 0.99
UPDATE_LAG = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Replay Buffer

In [4]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

# Dueling DQN

In [5]:
class DuelingDQN(nn.Module):
    def __init__(self, state_size, action_size, seed, type: int):
        super(DuelingDQN, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc_val = nn.Sequential(
            nn.Linear(state_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

        self.fc_adv = nn.Sequential(
            nn.Linear(state_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_size)
        )

        self.type = type

    def forward(self, x: torch.Tensor):
        val = self.fc_val(x)
        adv = self.fc_adv(x)

        if self.type == 1:
            return val + adv - adv.mean()
        else:
            return val + adv - adv.max()

In [6]:
class Agent:
    def __init__(self, state_size, action_size, seed):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        self.local = DuelingDQN(state_size, action_size, seed, 1).to(device)
        self.target = DuelingDQN(state_size, action_size, seed, 1).to(device)
        self.optimizer = optim.Adam(self.local.parameters(), lr=LR)

        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

        self.t_step = 0


    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        if len(self.memory) >= BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)

        self.t_step +=  1
        if self.t_step % UPDATE_LAG == 0:
            self.update_target()

    def update_target(self):
        self.target.load_state_dict(self.local.state_dict())

    def act(self, state, eps=0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.local.eval()
        with torch.no_grad():
            action_values = self.local(state)
        self.local.train()

        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences

        ''' Get max predicted Q values (for next states) from target model'''
        Q_targets_next = self.target(next_states).detach().max(1)[0].unsqueeze(1)

        ''' Compute Q targets for current states '''
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        ''' Get expected Q values from local model '''
        Q_expected = self.local(states).gather(1, actions)

        ''' Compute loss '''
        loss = F.mse_loss(Q_expected, Q_targets)

        ''' Minimize the loss '''
        self.optimizer.zero_grad()
        loss.backward()

        ''' Gradiant Clipping '''
        """ +T TRUNCATION PRESENT """
        for param in self.local.parameters():
            param.grad.data.clamp_(-1, 1)

        self.optimizer.step()


# REINFORCE

In [59]:
class Reinforce(nn.Module):
    def __init__(self, state_size, action_size, seed):
        super(Reinforce, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc = nn.Sequential(
            nn.Linear(state_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, action_size)
        )

    def forward(self, x: torch.Tensor):
        return F.softmax(self.fc(x), dim=1)

    def get_action(self, state: torch.Tensor):
        probs = self(state)
        probs = probs.squeeze(0)
        action = probs.multinomial(1)
        return action.data[0].item()

class Critic(nn.Module):
    def __init__(self, state_size, action_size, seed):
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.value_fc = nn.Sequential(
            nn.Linear(state_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
    def forward(self, x: torch.Tensor):
      return self.value_fc(x)

# Training loop

In [60]:
# train the dueling dqn


# state_size = env.observation_space.shape[0]
# action_size = env.action_space.n

def dqn(env, agent, n_episodes=10000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):

    scores_window = deque(maxlen=100)
    ''' last 100 scores for checking if the avg is more than 195 '''
    episode_score = []

    eps = eps_start
    ''' initialize epsilon '''

    for i_episode in range(1, n_episodes+1):
        state, _ = env.reset()
        score = 0
        for t in range(max_t):
            # print(state)
            action = agent.act(state, eps)
            # print(action)
            something = env.step(action)
            # print(something)
            next_state, reward, done, _trunc,  _ = something
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break


        scores_window.append(score)
        episode_score.append(score)

        eps = max(eps_end, eps_decay*eps)
        ''' decrease epsilon '''

        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")

        if i_episode % 100 == 0:
           print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=-100.0:
           print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
           break
    return episode_score

# begin_time = datetime.datetime.now()

# agent = Agent(state_size=state_size,action_size = action_size,seed = 0)
# dqn(env, agent)

# time_taken = datetime.datetime.now() - begin_time

# print(time_taken)

In [73]:
def reinforce(env, model, optimizer, n_episodes = 10000, max_t = 1000, baseline = False, critic = None, critic_optimizer = None):

    scores_window = deque(maxlen=100)
    for i_episode in range(n_episodes):
        state, _ = env.reset()
        states = []
        actions = []
        rewards = []
        score = 0
        for t in range(max_t):
            # print(state)
            state = torch.from_numpy(state).float().unsqueeze(0)
            action = model.get_action(state)
            # print(action)
            states.append(state)
            actions.append(action)
            next_state, reward, done, _trunc,  _ = env.step(action)
            state = next_state
            # print(reward)
            rewards.append(reward)
            score += reward
            if done:
                break

        scores_window.append(score)
        print('\rEpisode {}\tCurrent Score: {}\tAverage Score: {:.2f}'.format(i_episode, score, np.mean(scores_window)), end="")

        returns = []
        G = 0
        for s, a, r in zip(states[::-1], actions[::-1], rewards[::-1]):
            G = r + 0.99*G

            returns.append(G)
            # G.requires_grad = True
            # print(G.grad)
            # print(model(s)[0])
            # s = Variable(s)

        returns = torch.tensor(returns[::-1])
        # returns = (returns - returns.mean()) / (returns.std() + 1e-9)
        losses = []
        value_losses = []
        for s, a, G, r in zip(states, actions, returns, rewards):
            # prob, val = model(s)
            prob = model(s)[0][a].squeeze()
            val = 0
            if baseline:
              val = critic(s)
              # print(val[0].squeeze())
              td_error = (G - val[0].squeeze())
              value_losses.append(0.5 * td_error ** 2)
            # print(prob.requires_grad)

            loss = -torch.log(prob) * (G - val)
            losses.append(loss)

            # print(loss, G)
            # if loss < -100:
            #     print(loss)
            # # print(loss, prob)
            # optimizer.zero_grad()
            # loss.backward()
            # optimizer.step()
        loss = torch.stack(losses).sum()
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        # if baseline:
        #   value_loss.backward()
        optimizer.step()

        if baseline:
          value_loss = torch.stack(value_losses).sum()
          # print(value_losses)
          critic_optimizer.zero_grad()
          value_loss.backward()
          critic_optimizer.step()

        # print("One round done")

# Environments

In [77]:
env = gym.make("CartPole-v1")

In [79]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
model = Reinforce(state_size=state_size, action_size=action_size, seed=42)

optimizer = optim.Adam(model.parameters(), lr=LR)
critic = Critic(state_size = state_size, action_size=action_size, seed = 42)
critic_optimizer = optim.Adam(critic.parameters(), lr=LR)
begin_time = datetime.datetime.now()
reinforce(env, model, optimizer, baseline=True, critic = critic, critic_optimizer = critic_optimizer)
time_taken = datetime.datetime.now() - begin_time

print(time_taken)

Episode 175	Current Score: 361.0	Average Score: 171.69

KeyboardInterrupt: 